In [2]:
%gui wx
import sys
import os

#####################
# Import of utils.py functions
#####################
# Required to get utils.py and access its functions
notebook_dir = os.path.abspath("")
parent_dir = os.path.abspath(os.path.join(notebook_dir, '..'))
sys.path.append(parent_dir)
sys.path.append('.')
from utils import loadFSL, FSLeyesServer, mkdir_no_exist, interactive_MCQ

####################
# DIPY_HOME should be set prior to import of dipy to make sure all downloads point to the right folder
####################
os.environ["DIPY_HOME"] = "/home/jovyan/Data"


#############################
# Loading fsl and freesurfer within Neurodesk
# You can find the list of available other modules by clicking on the "Softwares" tab on the left
#############################
import lmod
await lmod.purge(force=True)
await lmod.load('fsl/6.0.7.4')
await lmod.load('freesurfer/7.4.1')
await lmod.list()

####################
# Setup FSL path
####################
loadFSL()

###################
# Load all relevant libraries for the lab
##################
import fsl.wrappers
from fsl.wrappers import fslmaths

import mne_nirs
import nilearn
from nilearn.datasets import fetch_development_fmri

import mne
import mne_nirs
import dipy
from dipy.data import fetch_bundles_2_subjects, read_bundles_2_subjects
import xml.etree.ElementTree as ET
import os.path as op
import nibabel as nib
import glob

import ants

import openneuro
from mne.datasets import sample
from mne_bids import BIDSPath, read_raw_bids, print_dir_tree, make_report


# Useful imports to define the direct download function below
import requests
import urllib.request
from tqdm import tqdm


# FSL function wrappers which we will call from python directly
from fsl.wrappers import fast, bet
from fsl.wrappers.misc import fslroi
from fsl.wrappers import flirt

# General purpose imports to handle paths, files etc
import glob
import pandas as pd
import numpy as np
import json
import subprocess

In [3]:
current_dir = os.path.abspath("")
print(f"current_dir: {current_dir}")
sys.path.append(current_dir)

dataset_id = 'ds000171'
subjects = ['sub-control{:02d}'.format(i+1) for i in range(20)]

dataset_path = os.path.join(current_dir, "data", dataset_id)
deriv_path = os.path.join(current_dir,"data", "derivatives")
preproc_path = os.path.join(deriv_path, 'preprocessed_data')

subject = "sub-control01"

mkdir_no_exist(dataset_path)
mkdir_no_exist(preproc_path)

current_dir: /data


In [5]:
fsleyesDisplay = FSLeyesServer()
fsleyesDisplay.show()

Overview of the brain before any preprocessing

In [14]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(op.join(dataset_path, subject, 'anat', '{}_T1w.nii.gz').format(subject))


(ipykernel_launcher.py:9312): Gdk-WARNING **: 23:17:40.266: gdkdrawable-x11.c:952 drawable is not a pixmap or window


1. Skull Removal

In [15]:
def get_skull_stripped_anatomical(bids_root, preproc_root, subject, robust=False):
    """
    Function to perform skull-stripping (removing the skull around the brain).
    This is a simple wrapper around the brain extraction tool (BET) in FSL's suite
    It assumes data to be in the BIDS format (which we will cover in the following weeks).
    The method also saves the brain mask which was used to extract the brain.

    The brain extraction is conducted only on the T1w of the participant.

    Parameters
    ----------
    bids_root: string
        The root of the BIDS directory
    preproc_root: string
        The root of the preprocessed data, where the result of the brain extraction will be saved.
    subject_id: string
        Subject ID, the subject on which brain extraction should be conducted.
    robust: bool
        Whether to conduct robust center estimation with BET or not. Default is False.
    """

    # For now all you need to do is that we remove the bones and flesh from the MRI to get the brain!
    anatomical_path = op.join(bids_root, subject, 'anat' , '{}_T1w.nii.gz'.format(subject))
    betted_brain_path = op.join(preproc_root, subject, 'anat')
    mkdir_no_exist(betted_brain_path)
    betted_brain_path = op.join(preproc_root, subject, 'anat', '{}_T1w'.format(subject))
    os.system('bet {} {} -m {}'.format(anatomical_path, betted_brain_path, '-R' if robust else ''))
    print("Done with BET.")

resulting_mask_path = op.join(preproc_path, subject, 'anat', '{}_T1w_mask.nii.gz'.format(subject))
get_skull_stripped_anatomical(dataset_path, preproc_path, subject, robust=True)

Done with BET.


In [16]:
fsleyesDisplay.resetOverlays()

fsleyesDisplay.load(op.join(dataset_path, subject, 'anat', '{}_T1w.nii.gz').format(subject))
fsleyesDisplay.load(resulting_mask_path)

In [20]:
def apply_fsl_mask(img_path, mask_path, masked_img_path):
    ###########################
    # Solution
    # By reading fslmaths documentation, one can see that the -mas option is exactly what we desire.
    ###########################
    os.system('fslmaths {} -mas {} {}'.format(img_path, mask_path, masked_img_path))

anatomical_path = op.join(dataset_path, subject, 'anat' , '{}_T1w.nii.gz'.format(subject))
betted_brain_path = op.join(preproc_path, subject, 'anat', '{}_T1w.nii.gz'.format(subject))# The brain without skull is in the derivatives folder
apply_fsl_mask(anatomical_path, resulting_mask_path, betted_brain_path)

In [21]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(betted_brain_path)

In [17]:
print("T")

T
